![astropy logo](https://astropy.readthedocs.io/en/stable/_static/astropy_banner.svg)

# Introduction to [`astropy`](https://astropy.readthedocs.io/en/stable/)

with [Brett Morris](http://brettmorr.is/)

**Dependencies**: astropy, astroquery, astroplan

### Outline
1. `astropy.units`
2. `astropy.time`
3. `astropy.coordinates`
4. `astropy.cosmology`
5. `astropy.table`
5. `astropy`-affiliated packages: `astroquery` & `astroplan`
6. `astropy.io.fits`
7. Exercises

*** 
 
## 1) [`astropy.units`](http://astropy.readthedocs.org/en/latest/units/): Problem sets are about to get easier

One of the modules most central to `astropy` is the `units` module, which will save you lots of time. 

In [ ]:
import astropy.units as u
import numpy as np

height = u.Quantity(1.778, unit=u.meter)
# or equivalently:
height = 1.778 * u.m

If you hate people, imperial units are supported:

In [ ]:
from astropy.units.imperial import foot
height.to(foot)

What is the light-travel time across one Brett? ($\Delta t = \Delta x / c$)

In [ ]:
from astropy.constants import c, m_e, h, M_sun, R_sun, R_earth, M_jup, R_jup, G

dt = height/c
dt

In [ ]:
G

Metric prefixes accepted (try `M` for mega, `p` for pico, etc.)

In [ ]:
dt.to(u.ns)

A quantity has two attributes: 

In [ ]:
dt.value, dt.unit

Vector quantities are where it's at: 

In [ ]:
distances = u.Quantity([1.3, 1.5, 1.7], unit=u.lightyear)

distances.to(u.m)

Quantities are either Python built-in types (float, int) or numpy arrays with metadata. To get at the underlying numbers, use the `value` attribute: 

In [ ]:
distances.value, type(distances.value)

If the result of what you're computing is unitless, but you arrived there by combining lots of units, then you might need to use the `float` function to turn your unitful answer into a dimensionless quantity by resolving all of the unit equivalences: 

In [ ]:
expansion_rate = 67 * u.km / u.s / u.Mpc
duration = 1 * u.Gyr

expansion_rate * duration

There are a bunch of useful quantities stored in `astropy.constants`, which will save you frustration in problem sets: 

In [ ]:
from astropy.constants import R_sun, R_earth, R_jup
from astropy.constants import M_sun, M_earth, m_e
from astropy.constants import G, h, k_B

print(M_sun)

You can use these constants like units:

In [ ]:
# Calculate the black hole mass in units of solar masses: 
black_hole_mass = 12e31 * u.kg

black_hole_mass.to(M_sun)

The above result should be read as "60 (solar masses)". To see the quantity without it's unit, use `value`: 

In [ ]:
black_hole_mass.to(M_sun).value

***

## 2) [`astropy.time`](http://astropy.readthedocs.org/en/latest/time/index.html): Time objects for humans

There are many distinct and confusing time systems used in astronomy, and the `astropy.time` module provides a convenient means of translating between them – never code your own JD-to-ISO time converter or try to remember whether or not the difference between JD and MJD has a 0.5 in it again!

In [ ]:
# The astropy.time.Time object contains a time in a specified format
from astropy.time import Time

# If the input format is not specified, it will guess. Here's an ISO formatted string:
Time('2020-01-01 12:34:56')

Here's a Julian Date:

In [ ]:
t = Time(2459372.0242592595, format='jd')
t

Convert between time formats by calling `t.iso`, `t.mjd`, etc.

In [ ]:
t.iso

By default, the scale (or time standard) is set to **UTC**, which is defined to keep an integer number of seconds per day. There are other time standards like **UT1** which are defined by the rotation of the Earth (see [my blog post on time standards](http://bmmorris.blogspot.com/2015/06/ut1-utc-and-astropy.html) for more background). Converting between the two can be messy, but not with astropy:

In [ ]:
print('Available time scales: {0}'.format(', '.join(Time.SCALES)))
t.scale

In [ ]:
t.ut1

If converting between UTC and UT1 you raises an `IndexError` like this, 
```
IndexError: (some) times are outside of range covered by IERS table.
```
it's because you need more up-to-date Earth rotation data since the Earth's rate of rotation is constantly changing. See the `astropy.time` docs on [Transformation offsets](http://astropy.readthedocs.org/en/stable/time/index.html#transformation-offsets) to update your Earth rotation data.

In [ ]:
t.ut1.iso

Lastly, arrays of times can be generated from numpy arrays:

In [ ]:
Time.now()

In [ ]:
times = Time.now() + np.linspace(0, 1, 10)*u.year

In [ ]:
times

`Time` objects are also great for plotting a time series. For exampel, try using the `plot_date` attribute with `plt.plot_date`, or `decimalyear` with `plt.plot`: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

times = Time.now() - np.linspace(0, 10, 100) * u.year
fluxes = 0.01 * np.random.randn(len(times)) + 1

plt.plot(times.decimalyear, fluxes)
plt.xlabel('Date')
plt.ylabel('Flux')
plt.show()

***


## 3) [`astropy.coordinates`](http://astropy.readthedocs.org/en/latest/coordinates/index.html)

![XKCD comic](http://imgs.xkcd.com/comics/standards.png)

Convert the position of your target from one coordinate system to another without opening a reference book!

Let's define the galactic center in the natural coordinate system:

In [ ]:
from astropy.coordinates import SkyCoord

gal_center = SkyCoord(l=0*u.deg, b=0*u.deg, frame='galactic')
print(gal_center)

Now let's say you have to tell an observer where that is in ICRS coordinates: what is that position in RA/Dec?

In [ ]:
gal_center.icrs

You can resolve targets by name with the `from_name` class method

In [ ]:
sgr_a = SkyCoord.from_name('Sgr A*')
print(sgr_a)

Let's represent these coordinates in various formats with `.degree`, `.hourangle`:

In [ ]:
sgr_a.ra.degree

and experiment with the string outputs you'd use in a proposal, like `dms`, `hmsdms`, `decimal`:

In [ ]:
sgr_a.to_string(style='hmsdms', sep=':')

With a specified location on Earth, you can compute alt/az coordinates for any `SkyCoord`

In [ ]:
from astropy.coordinates import EarthLocation, AltAz

# Define Earth location:
longitude, latitude, elevation = (-122.3331*u.deg, 47.6097*u.deg, 0*u.m)
seattle = EarthLocation.from_geodetic(longitude, latitude, elevation)

# Define alt/az frame:
alt_az_frame = AltAz(obstime=Time.now(), location=seattle)

# Transform the coordinate to the new reference frame, and print
sgr_a_altaz = sgr_a.transform_to(alt_az_frame)
sgr_a_altaz.to_string(style='hmsdms')

***


## 4) [`astropy.cosmology`](http://astropy.readthedocs.org/en/latest/cosmology/): No more JavaScript cosmology calculators for you!

First, choose a cosmology (e.g.: `Planck15`, `WMAP9`) and get $H_0$:

In [ ]:
from astropy.cosmology import WMAP9 as cosmo

cosmo.H(z=0)

In [ ]:
cosmo.angular_diameter_distance(z=1)

In [ ]:
cosmo.luminosity_distance(z=1)

In cosmology class you'll still have to learn to solve these from scratch, but you can double check yourself like so: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

z = np.linspace(0, 10, 50)

# Compute some parameters
t_lookback = cosmo.lookback_time(z)
T_cmb = cosmo.Tcmb(z)
D_A = cosmo.angular_diameter_distance(z)
D_L = cosmo.luminosity_distance(z)

fig, ax = plt.subplots(2, 2, figsize=(10, 10))

ax[0, 0].plot(z, t_lookback)
ax[0, 0].set(title='Look-back Time', xlabel='$z$', 
             ylabel=r'$t_{{lookback}}(z)$ [{0}]'.format(t_lookback.unit))

ax[0, 1].plot(z, T_cmb)
ax[0, 1].set(title='$T_{CMB}$', xlabel='$z$', 
             ylabel='$T_{{CMB}}(z)$ [{0}]'.format(T_cmb.unit))

ax[1, 0].plot(z, D_A)
ax[1, 0].set(title='Angular Diameter Distance', xlabel='$z$', 
             ylabel=r'$D_A(z)$ [{0}]'.format(D_A.unit))

ax[1, 1].plot(z, D_L)
ax[1, 1].set(title='Luminosity Distance', xlabel='$z$', 
             ylabel=r'$D_L(z)$ [{0}]'.format(D_L.unit))

fig.subplots_adjust(wspace=0.3)
fig.suptitle('Cosmology: {0}'.format(cosmo.name), fontsize=18);

# 5) `astropy.table`: Table objects for physical scientists

What makes a table object specific to physical scientists, you ask? **Units** my friend, units. In general, [pandas](http://pandas.pydata.org) has the most mature table-like data structures in Python, but the astropy table is nifty, so let's see how it works. 

A table can be constructed a bunch of ways. Let's initialize one from an array: 

In [ ]:
from astropy.table import Table

example_data = np.random.randint(0, 100, 50).reshape((10, 5))
column_names = ['a', 'b', 'c', 'd', 'e']

table = Table(example_data, names=column_names)

table

As you can see, the astropy table has some special powers inside iPython notebooks, and gets rendered nicely. 

The first row tells you the names of each column. You can access a column of data from a table by treating the table like a dictionary: 

In [ ]:
table['a']

This column object has a `.data` attribute which you can use to get at the `numpy` array underneath:

In [ ]:
table['a'].data

This gets at the heart of what an astropy table is. It's essentially an ordered dictionary of columns. Each column is a numpy array _with metadata_. That metadata is what makes the table useful, because those columns, for example, can have units! 

In [ ]:
table['a'].unit = u.km
table['b'].unit = u.lightyear
table['c'].unit = u.kg
table['d'].unit = u.s
table['e'].unit = u.Mpc

table

Now you can do operations on each column as though it was a unit vector:

In [ ]:
table['b'].to(u.pc)

and you can get each element as a quantity, or not: 

In [ ]:
# Not a quantity:
table['a'][2], type(table['a'][2])

In [ ]:
# A quantity:
table['a'].quantity[2]

## Reading and writing ascii text tables
 
Perhaps a collaborator will send you some IDL-generated text tables to work with, and you'll want to open it with Python. Sometimes the easiest way to do this will be with `astropy.table.Table`. Let's create an example table in the cell below: 

In [ ]:
%%writefile example_table.txt
a  b  c
0  2  2
3  4  6
34 4  1
6  36 5
86 7  3

In [ ]:
from astropy.table import Table

table = Table.read('example_table.txt', format='ascii')

table

Well that was easy! We didn't have to specify anything, and it did most of the work. We can now take this `astropy.table.Table` object and make it even more useful, by giving the columns units, etc.:

In [ ]:
table['a'].unit = u.kg

table

We can write this table out using the [very very flexible `ascii.write` function](http://docs.astropy.org/en/v0.2.1/io/ascii/index.html): 

In [ ]:
# Directly output a table in LaTeX format: 
table.write('latex_table.tex', format='ascii.latex')

# Output a CSV file: 
table.write('csv_table.csv', format='csv')

# Write a table with the column names at the top, in a comment
table.write('table_with_header.txt', format='ascii.commented_header')

***


## 6a) Affiliated Package: [`astroquery`](http://astroquery.readthedocs.org)

Since `astropy` is a collection of fundamental tools that are easy to use, lots of packages have been built on top of `astropy`, but not necessarily merged into `astropy` core. One of those is `astroquery`, which allows you to query astronomical databases with ease.

Let's query for the SIMBAD entry for a planet hosting star, HD 189733:

In [ ]:
from astroquery.simbad import Simbad

Simbad.query_object('HD 189733')

Let's query Vizier for the famous list of standard stars from [Landolt (1992)](http://adsabs.harvard.edu/abs/1992AJ....104..340L). The [`astropy.table`](http://astropy.readthedocs.org/en/latest/table/) that is returned to you will have the same information as [this Vizier query page](http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=II/183A/table2).

In [ ]:
from astroquery.vizier import Vizier

landolt_table = Vizier.get_catalogs('Landolt 1992')[0]
landolt_table

## 6b) Affiliated Package: [`astroplan`](https://astroplan.readthedocs.org/en/latest/)

`astroplan` is an `astropy`-affiliated package that helps you calculate when objects are observable. Here's a quick example for determining which targets are visible right now from Apache Point Observatory:

In [ ]:
from astroplan import Observer, FixedTarget

# Targets are stored as `astroplan.FixedTarget` objects
target_names = ['Polaris', 'Sirius', 'Vega', 'Rigel']
targets = [FixedTarget.from_name(target) for target in target_names]

# Observatories are `astroplan.Observer` objects
observatory = Observer.at_site("Apache Point")

# Which targets are visible right now?
observatory.target_is_up(Time.now(), targets)

Now let's see which of those targets are visible over a time range of the next ten days, given the following constraints: 

* Observations must occur between civil twilights
* The altitude of the target must be $20^\circ < $alt$ < 85^\circ$

In [ ]:
from astroplan import AtNightConstraint, AltitudeConstraint, observability_table

time_range = Time.now() + np.array([0, 10])*u.day
constraints = [AtNightConstraint.twilight_civil(),
               AltitudeConstraint(min=20*u.deg, max=85*u.deg)]

observability_table(constraints, observatory, targets, time_range=time_range)

Let's track that target's motion through the sky for the next ten hours in a plot: 

In [ ]:
from astroplan.plots import plot_sky

# Plot at times: 
plot_times = Time.now() + np.linspace(0, 10, 10)*u.hour

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    plot_sky(targets[2], observatory, plot_times)

For a more detailed run through of astroplan, [here's another notebook to check out](https://gist.github.com/bmorris3/19374760eb11271850ec).


# 7) `astropy.io.fits`: Reading and writing FITS files

Astronomers (unfortunately) use FITS files a lot, so let's practice using FITS files with astropy. The following command will download a FITS image for us to work with:

In [ ]:
from astropy.utils.data import download_file

url = 'https://fits.gsfc.nasa.gov/samples/WFPC2ASSNu5780205bx.fits'
example_fits_path = download_file(url)

We can read in FITS files in two ways. The first is generic, and allows us to see the different extensions. 

In [ ]:
from astropy.io import fits

hdus = fits.open(example_fits_path)

print(hdus)

first_hdu = hdus[0]

plt.imshow(first_hdu.data, cmap=plt.cm.Greys)
plt.title(first_hdu.header['OBJECT'])
plt.show()

Alternatively, if you know what HDU you want to access, and you want just the header or the data, you can use the following: 

In [ ]:
image = fits.getdata(example_fits_path)
header = fits.getheader(example_fits_path)

You can access particular header cards from the `astropy.io.fits.header.Header` object like a dictionary: 

In [ ]:
date_obs = header['DATE-OBS']
exp_time = header['EXPTIME']

print("Observation time: {0}".format(date_obs))
print("Exposure duration: {0} s".format(exp_time))

To see the available keywords within the header, do:

In [ ]:
list(header.keys())[:10]

If you want to write some results to a FITS file, you can do so like this: 

In [ ]:
# Create a 2D, 10 x 10 random number array: 
example_data = np.random.randn(100).reshape((10, 10)) 

fits.writeto('example_data.fits', example_data, header=header, overwrite=True)

***

# 8) Exercises

**1)** Get the light travel time to the sun in minutes, given it's distance *right now* (hint: check out [`astropy.coordinates.get_sun`](http://astropy.readthedocs.org/en/latest/api/astropy.coordinates.get_sun.html?highlight=get_sun#astropy.coordinates.get_sun)).

**2)** Using your current distance from the Sun in #1, calculate which is greater: the force of gravity between you and the Sun right now, or between you and a bowling ball-sized chunk of neutron star placed 12 kilometers away. 

Let's assume your mass is 60 kg. Use `astropy.constants` to get the gravitational constant $G$ and the mass of the sun $M_\odot$. Let's say bowling balls have $r \sim 22$ cm, and neutron stars have a density of $\rho \sim  3.7 \times 10^{17} $kg m$^{-3}$.

**3)** Calculate the Schwarzschild radius in units of solar radii of the Sgr A*, the Milky Way's supermassive black hole with $M = 4.31 \times 10^6 M_\odot$, given

$$r_\mathrm{s} = \frac{2 G M}{c^2}$$

and the distance to the galactic center $d_{center} = 7.94$ kpc. Also calculate the angular size of the event horizon on the sky in microarcseconds.

**4)** Represent your birthday in the following time formats: ISO, JD, MJD and decimal year, all with the UTC time standard (default).

**5)** Using the table of Landolt standards which we generated above (`landolt_table`), find the name of the star with the brightest _V_ magnitude (smallest number), and find its position in galactic coordinates (hint: [`SkyCoord` docs](http://docs.astropy.org/en/stable/coordinates/#transformation)). 